<a href="https://colab.research.google.com/github/lacykaltgr/ait-assessments/blob/main/06_AIT_CNN_CIFAR10_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Copyright

<PRE>
Copyright (c) 2023 Bálint Gyires-Tóth - All Rights Reserved
You may use and modify this code for research and development purpuses.
Using this code for educational purposes (self-paced or instructor led) without the permission of the author is prohibited.
</PRE>

# Convolutional Neural Network exercise

In this notebook, you will build a convolutional neural network, which offers superior results over the previous MLP solution. Additionally, you will perform a more detailed evaluation.  

In [12]:
# imports 
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Activation
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping

# 1. Data
The dataset we will use is the well-known CIFAR10 (Canadian Institute For Advanced Research). Explore the details on the Keras website first: https://keras.io/api/datasets/cifar10/

There isn't much difference between the data preprocessing part and the one we did before. The only difference is that we don't have to reshape images into vectors, but we are working with a 4 dimensional data structure, where the dimensions are: batch, X, Y, and color channels. 

In the event that anything is not clear regarding the data preparation part, please check the previous CIFAR10 notebook.

In [2]:
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
# it is in int8 format, the neural network requires float32
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

In [4]:
train_ratio  = 0.8
train_length = X_train.shape[0]
train_split  = int(train_ratio*train_length)
X_valid, Y_valid = X_train[train_split:], Y_train[train_split:]
X_train, Y_train = X_train[:train_split], Y_train[:train_split]

In [5]:
mean = X_train.mean(axis=0)
std  = X_train.std(axis=0)

X_train = (X_train-mean)/std
X_valid = (X_valid-mean)/std
X_test  = (X_test-mean)/std

In [6]:
nb_classes = len(np.unique(Y_train))
Y_train = to_categorical(Y_train, nb_classes)
Y_valid = to_categorical(Y_valid, nb_classes)
Y_test  = to_categorical(Y_test, nb_classes)

In [7]:
print("Shapes of the training, validation and test input data:", X_train.shape, X_valid.shape, X_test.shape)
print("Shapes of the training, validation and test output data:", Y_train.shape, Y_valid.shape, Y_test.shape)
print("Mean values of the training, validation and test input data:", X_train.mean(), X_valid.mean(), X_test.mean())
print("Standard deviation of the training, validation and test input data:", X_train.std(), X_valid.std(), X_test.std())

Shapes of the training, validation and test input data: (40000, 32, 32, 3) (10000, 32, 32, 3) (10000, 32, 32, 3)
Shapes of the training, validation and test output data: (40000, 10) (10000, 10) (10000, 10)
Mean values of the training, validation and test input data: 2.2788842e-09 0.0023437198 0.013331206
Standard deviation of the training, validation and test input data: 1.0000023 0.9965626 0.99783075


# 2. Exercise: training

Create and train a convolutional neural network with the following requirements:
* apply early stopping with patience=5 and monitor the validation accuracy, don't forget to load back the best weights after early stopping
* the number of trainable parameters is lower than 200.000
* the validation accuracy must be higher than 72%

Hints:
* you can always inspect the number of trainable parameters per layer with model.summary()
* usually the most trainable parameters are between the last convolutional layer and the next dense layer -- so you should have a reasonable output size at the last convolutional layer
* regularization helps (e.g. DropOut and/or BatchNormalization)
* use the right activation function at the output layer and the corresponding loss function
* for the chosen activation functions, please use the appropriate initialization method
* you can match the dimension of the last conv/pooling layer to the next dense layer with e.g. Flatten()
* ADAM optimizer is a reasonable choice 
* don't forget to turn on GPU support in Colab, otherwise, it would be quite slow to train the networks



In [69]:
es = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True) # early stopping

In [47]:
X_train.shape[1:]

(32, 32, 3)

In [156]:
# model definition
model = Sequential()
model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(X_train.shape[1:])))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.15))
model.add(Conv2D(32, (2,2), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.15))
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.15))

model.add(Flatten())

model.add(BatchNormalization())
model.add(Dense(64, kernel_initializer=HeNormal()))
model.add(Dropout(0.4))
model.add(Dense(32, kernel_initializer=HeNormal()))
model.add(Dense(10, kernel_initializer=HeNormal(), activation='softmax'))

# loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [93]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_73 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d_71 (MaxPoolin  (None, 16, 16, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_74 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_72 (MaxPoolin  (None, 8, 8, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_75 (Conv2D)          (None, 8, 8, 64)          36928     
                                                                 
 max_pooling2d_73 (MaxPoolin  (None, 4, 4, 64)       

Please, check the number of trainable paramers:

In [61]:
print("#trainable parameters:",model.count_params())

#trainable parameters: 134858


In [157]:
# training
network_history = model.fit(X_train, Y_train, 
                            validation_data=(X_valid, Y_valid), 
                            batch_size=64, 
                            epochs=200, 
                            callbacks=[es])

Epoch 1/200
625/625 [==============================] - 9s 9ms/step - loss: 1.8359 - val_loss: 1.5065
Epoch 2/200
625/625 [==============================] - 5s 7ms/step - loss: 1.3168 - val_loss: 1.1709
Epoch 3/200
625/625 [==============================] - 5s 8ms/step - loss: 1.1921 - val_loss: 1.0777
Epoch 4/200
625/625 [==============================] - 5s 7ms/step - loss: 1.1086 - val_loss: 0.9715
Epoch 5/200
625/625 [==============================] - 5s 8ms/step - loss: 1.0512 - val_loss: 0.9276
Epoch 6/200
625/625 [==============================] - 5s 8ms/step - loss: 1.0059 - val_loss: 0.8926
Epoch 7/200
625/625 [==============================] - 4s 7ms/step - loss: 0.9635 - val_loss: 0.8950
Epoch 8/200
625/625 [==============================] - 5s 8ms/step - loss: 0.9324 - val_loss: 0.8335
Epoch 9/200
625/625 [==============================] - 5s 7ms/step - loss: 0.9132 - val_loss: 0.8262
Epoch 10/200
625/625 [==============================] - 5s 8ms/step - loss: 0.8920 - val_lo

# 3. Exercise: evaluation on test data
You will evaluate the classification of the test data using the common metrics and the confusion matrix in this section. 


As a first stes, let's run the predictions and convert the predicted softmax outputs to dense labels. The one-hot encoded labels are also converted back to dense labels, as they are required for the evaluation functions.

In [158]:
probas = model.predict(X_test)
preds  = np.argmax(probas,axis=1)
Y_test_dense = np.argmax(Y_test, axis=1) # get the original dense labels of the test data

313/313 [==============================] - 1s 2ms/step


## 3.1. Exercise: Metrics
please review the common classification metrics (accuracy, precision, recall, F1) for the complete model (not per class). 

Hints:
* use macro averaging
* you have to import the right functions from [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html)


In [77]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [159]:
print("Accuracy:", accuracy_score(Y_test_dense, preds))
print("Precision:", precision_score(Y_test_dense, preds, average='macro'))
print("Recall:", recall_score(Y_test_dense, preds, average='macro'))
print("F1:", f1_score(Y_test_dense, preds, average='macro'))

Accuracy: 0.7708
Precision: 0.7711782498540275
Recall: 0.7707999999999999
F1: 0.7697323966103156



## 3.2. Exercise: Confusion matrix
Calculate the confusion matrix and display the results in a heatmap (with decimal values).

Hints:
* you have to do the imports this time too, e.g. from [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) and the [heatmap function](https://seaborn.pydata.org/generated/seaborn.heatmap.html) of [seaborn](https://seaborn.pydata.org/)
* for displaying the decimal values, you have to turn annot=True and define fmt='d' in the heatmap function of seaborn

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
conf = <TODO>
plt.figure(figsize=(7,6))
sns.heatmap(<TODO>)